In [37]:
#import Google earth engine
import ee
import geemap

In [38]:
#trigger the authentication flow
ee.Authenticate()

Enter verification code: 4/1AfJohXnYxcbOMzTChSPeHR2ozT21oHpJbPe9OYt6erOj2A2qBvEVRfStR6A

Successfully saved authorization token.


In [39]:
#initialize the library
ee.Initialize()

In [41]:
Map= geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [53]:
#Area of Interest for Supervised Classification
point= ee.Geometry.Point([-122.4439, 37.7538])

#Loading LANDSAT 8 Data
image= ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
.filterDate('2016-01-01','2016-12-01')\
.filterBounds(point)\
.sort('CLOUD_COVER')\
.first()\
.select('B[1-7]')

#defining visualization parameters
visParams= {
    'bands':['B5', 'B4', 'B3'],
    'min': 0,
    'max': 3000
}

In [54]:
Map.centerObject(point,8)
Map.addLayer(image, visParams, 'LANDSAT-8')

In [55]:
#Getting image properties
ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()


'2016-11-18'

In [56]:
image.get('CLOUD_COVER').getInfo()

0.08

In [58]:
# training dataset
nlcd= ee.Image('USGS/NLCD/NLCD2016').select('landcover').clip(image.geometry())
Map.addLayer(nlcd, {}, 'NLCD')
Map


Map(bottom=25636.0, center=[37.75379999999999, -122.44390000000001], controls=(WidgetControl(options=['positio…

In [60]:
#Making training dataset
points= nlcd.sample(**{
    'region': image.geometry(),
    'numPixels': 5000,
    'scale':30,
    'seed': 0,
    'geometries': True
})
Map.addLayer(points, {}, 'training', False)

In [61]:
#variables for classifying
bands=['B1','B2','B3','B4','B5','B6','B7']
label='landcover'


In [62]:
#Overlaying the training set on imagery to get training 
training= image.select(bands).sampleRegions(**{
    'collection': points,
    'properties': [label],
    'scale': 30
})

In [63]:
#Training a CART classifier with default parameters
trained= ee.Classifier.smileCart().train(training, label, bands)



In [65]:
print(training.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'B1': 575, 'B2': 814, 'B3': 1312, 'B4': 1638, 'B5': 1980, 'B6': 2091, 'B7': 1967, 'landcover': 31}}


In [66]:
#Classifying the image
result= image.select(bands).classify(trained)

In [68]:
Map.addLayer(result.randomVisualizer(),{}, 'classified')
Map

Map(bottom=25636.0, center=[37.75379999999999, -122.44390000000001], controls=(WidgetControl(options=['positio…

In [69]:
#Rendering categorical map
class_values= nlcd.get('landcover_class_values').getInfo()
class_values

[11,
 12,
 21,
 22,
 23,
 24,
 31,
 41,
 42,
 43,
 51,
 52,
 71,
 72,
 73,
 74,
 81,
 82,
 90,
 95]

In [70]:
class_palette= nlcd.get('landcover_class_palette').getInfo()
class_palette

['476ba1',
 'd1defa',
 'decaca',
 'd99482',
 'ee0000',
 'ab0000',
 'b3aea3',
 '68ab63',
 '1c6330',
 'b5ca8f',
 'a68c30',
 'ccba7d',
 'e3e3c2',
 'caca78',
 '99c247',
 '78ae94',
 'dcd93d',
 'ab7028',
 'bad9eb',
 '70a3ba']

In [74]:
landcover= result.set('classification_class_values', class_values)
landcover= landcover.set('classification_class_palette', class_palette)

In [75]:
Map.addLayer(landcover, {}, 'Land Cover')
Map

Map(bottom=25636.0, center=[37.75379999999999, -122.44390000000001], controls=(WidgetControl(options=['positio…

In [78]:
#Visualizing the result
cluster_layer=Map.layers[-1]
cluster_layer.interact(opacity=(0,1,0.1))

Box(children=(FloatSlider(value=1.0, description='opacity', max=1.0),))

In [80]:
#adding legend
Map.add_legend(builtin_legend='NLCD')
Map

Map(bottom=50996.0, center=[37.70120736474139, -122.43160979208528], controls=(WidgetControl(options=['positio…